In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [6]:
path_risks = os.path.join(os.getcwd(), 'data/risk_sim.csv')
path_supplychain = os.path.join(os.getcwd(), 'data/supplychain.csv')

risks = pd.read_csv(path_risks, sep=';')

In [3]:
from backend.company import Company
from backend.risk import Risk
from backend.supplychain import SupplyChain
from backend.types.risk_type import RiskType

In [4]:
company_list = []

for companies in risks['Companies']:
    for company in companies:
        if company.isalpha() and company not in company_list:
            company_list.append(company)

print(list(company_list))

company_obj_list = []

for company in company_list:
    risks_company = []
    for i, affected_companies in enumerate(risks['Companies'], 0):
        if company in affected_companies:
            row = risks.iloc[i]
            impacts = {'delivery assurance': row['delivery assurance'],
                           'customer requirements': row['customer requirements'],
                           'product data': row['product data'],
                           'green conversion': row['green conversion']}
            new_risk = Risk(row['Risks'], row['Disruption'], row['Frequency'], impacts)
            risks_company.append(impacts)

    company_obj = Company(risks_company)
    company_obj_list.append(company_obj)

supply_chain = SupplyChain(company_obj_list)




['a', 'b', 'c']


In [48]:
company_supply = pd.read_csv(path_supplychain, sep=';')

company_hierarchy = []
used_companies = []

for k in range(len(company_supply.index)):
    level_companies = []
    level_used_companies = []
    for i, company in enumerate(company_supply['Company'], 0):
        suppliers = company_supply.iloc[i]['Supplier'].replace(" ", "").split(',')

        if company not in used_companies and company not in level_used_companies:
            if 'origin' in suppliers:
                level_used_companies.append(company)
                level_companies.append({'company': company, 'suppliers': suppliers})
            elif all(x in used_companies and x not in level_companies for x in suppliers):
                level_used_companies.append(company)
                level_companies.append({'company': company, 'suppliers': suppliers})

    used_companies = used_companies + level_used_companies

    company_hierarchy.append(level_companies)

company_hierarchy = [level for level in company_hierarchy if level != []]

print(company_hierarchy)

[[{'company': 'a', 'suppliers': ['origin']}], [{'company': 'b', 'suppliers': ['a']}], [{'company': 'c', 'suppliers': ['b', 'a']}, {'company': 'g', 'suppliers': ['a', 'b']}, {'company': 'aa', 'suppliers': ['a', 'b']}], [{'company': 'd', 'suppliers': ['a', 'c']}], [{'company': 'e', 'suppliers': ['a', 'd']}], [{'company': 'f', 'suppliers': ['e', 'd', 'b']}]]
